In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
lib = pd.read_csv('Pfam-A.regions.uniprot.tsv', sep='\t')

In [3]:
Nc7 = pd.read_csv('Nc7tmp.csv')
lwf = pd.read_csv('lwftmp.csv')

In [8]:
# get domian info and uniport acc from database
Nc7 = Nc7.merge(lib, how='left', on='uniprot_acc')

In [9]:
Nc7.to_csv('Nc7tmp.csv', index=False)

In [52]:
df = Nc7

df = df[df.apply(lambda x: x.Pos >= x.seq_start and x.Pos <= x.seq_end, axis=1)]
df['query'] = df.apply(lambda x: x.wt_seq[int(x.seq_start-1):int(x.seq_end)], axis=1)
onDomain = df

<ipython-input-52-ad9ff81ea367>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['query'] = df.apply(lambda x: x.wt_seq[int(x.seq_start-1):int(x.seq_end)], axis=1)


In [53]:
onDomain

,gene,raw,Pos,wt,mt,uniprot_acc,Length,wt_seq,seq_version,crc64,md5,pfamA_acc,seq_start,seq_end,query
0,caiD,p.Met217Ile,217,M,I,P31551,261,MSESLHLTRNGSILEITLDRPKANAIDAKTSFEMGEVFLNFRDDPQ...,4,2C4DD6C3D16995CC,d9c8f73871e078c879e4ff865e64a61d,PF00378,8,261,TRNGSILEITLDRPKANAIDAKTSFEMGEVFLNFRDDPQLRVAIIT...
1,caiT,p.Leu178Val,178,L,V,P31553,504,MKNEKRKTGIEPKVFFPPLIIVGILCWLTVRDLDAANVVINAVFSY...,2,A852310DBEA22D82,38b4db6538bce110ff050407026631c8,PF02028,14,501,VFFPPLIIVGILCWLTVRDLDAANVVINAVFSYVTNVWGWAFEWYM...
2,kefC,p.His502Arg,502,H,R,P03819,620,MDSHTLIQALIYLGSAALIVPIAVRLGLGSVLGYLIAGCIIGPWGL...,2,9995B2E8E3C1DCE3,05e52805c257ab46aedd76f2937d2a46,PF02254,402,514,VIIAGFGRFGQITGRLLLSSGVKMVVLDHDPDHIETLRKFGMKVFY...
6,yagH,p.Pro303Ser,303,P,S,P77713,536,MEITNPILTGFNPDPSLCRQGEDYYIATSTFEWFPGVRIYHSRDLK...,1,60D3F038215CD216,481f72f8536242029e9c5cab2d79ffc8,PF04616,3,303,ITNPILTGFNPDPSLCRQGEDYYIATSTFEWFPGVRIYHSRDLKNW...
7,betB,p.Ser459Phe,459,S,F,P17445,490,MSRMAEQQLYIHGGYTSATSGRTFETINPANGNVLATVQAAGREDV...,3,747F822DA1233808,49fcbbfa7b0d959af14afbe967b8c19b,PF00171,15,479,YTSATSGRTFETINPANGNVLATVQAAGREDVDRAVKSAQQGQKIW...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,yiaF,p.Asp85Asn,85,D,N,P0ADK0,236,MATGKSCSRWFAPLAALLMVVSLSGCFDKEGDQRKAFIDFLQNTVM...,2,B7268DDA83127AE5,d7997edec107c0e6dafb6563581aac7f,PF11254,14,232,LAALLMVVSLSGCFDKEGDQRKAFIDFLQNTVMRSGERLPTLTADQ...
308,yhfT,p.Asp315Ala,315,D,A,P45546,434,MDLYIQIIVVACLTGMTSLLAHRSAAVFHDGIRPILPQLIEGYMNR...,2,8AEE6E009E7A9D14,b9e8ce3771e4cd3d3122371a39c6e97f,PF10797,6,431,QIIVVACLTGMTSLLAHRSAAVFHDGIRPILPQLIEGYMNRREAGS...
309,crp,p.Lys29Thr,29,K,T,P0ACJ8,210,MVLGKPQTDPTLEWFLSHCHIHKYPSKSTLIHQGEKAETLYYIVKG...,1,DCBC24FA46C61B3D,9a58754a5c561c0784b75360a962f090,PF00027,21,110,IHKYPSKSTLIHQGEKAETLYYIVKGSVAVLIKDEEGKEMILSYLN...
311,yjfZ,p.Gly149Ser,149,G,S,P39308,264,MTLPTTIYSFPAYLSRFSSTDKPVKLKFHQYARATLLSNRGRDHNC...,1,28487344D5A4E652,ce8b68e5a0fc40ff61e65e961a41ee96,PF10887,1,264,MTLPTTIYSFPAYLSRFSSTDKPVKLKFHQYARATLLSNRGRDHNC...


## Generate scripts to run jackhmmer

In [1]:
hmdir = 'hm_on_domain/'
def getSeq(x):
    start = int(x['seq_start'])
    end = int(x['seq_end'])
    seq = x['query']
    start += 1
    if os.path.exists(hmdir+x['gene']+'_'+str(start)):
        return
    os.mkdir(hmdir+x['gene']+'_'+str(start))
    with open(hmdir+x['gene']+'_'+str(start)+'/query.fa', 'w') as f:
        f.write('>'+x['gene']+'\n')
        f.write(seq)
    print('cd /work/'+x['gene']+'_'+str(start))
    print('pwd')
    n = 2
    l = len(seq)/2
    if l > 300:
        n = 2
    n = str(n)
    l = str(l)
    with open(hmdir+x['gene']+'_'+str(start)+'/run_jackhmmer.sh', 'a') as f:
        f.write('jackhmmer -N '+n+' -o log -A msa.txt  --noali --notextw -T '+l+' --domT '+l+' --incT '+l+' --incdomT '+l+' --cpu 8 query.fa /work/uniref100.fasta\n')

onDomain.apply(getSeq, axis=1)

In [14]:
g2p = pd.read_csv('gene2Pid.tab', sep='\t')
gene = g2p.Gene.values
entry = g2p.Entry.values
gs = []
es = []
g2p = {}
for i in range(0, 643):
    if gene[i] not in gs:
        gs.append(gene[i])
        es.append(entry[i])
g2p = pd.DataFrame({'Gene':gs, 'uniprot_acc':es})

In [17]:
df = g2p.merge(lib, how='left', on='uniprot_acc')
g2p = df
df = pd.read_csv('SRR030258.se.vcf.csv')

In [36]:
def getPos(x):
    gene = x['Gene']
    info = g2p[g2p.Gene == gene]
    s = info.seq_start.values.tolist()
    e = info.seq_end.values.tolist()
    for i in range(len(s)):
        if x['Pos'] >= s[i] and x['Pos'] <= e[i]:
            return int(s[i])
    return 1

In [37]:
df['start'] = df.apply(getPos, axis=1)

In [39]:
df.to_csv('SRR030258.se.vcf.csv', index=False)

In [62]:
df = pd.read_csv('SRR030258.se.vcf.csv')

In [63]:
fa = {}
seq = ''
gn = 'na'
with open('genes.fasta') as f:
    for line in f.readlines():
        if line[0] == '>':
            fa[gn] = seq
            gn = line.split('GN=')[1].split(' ')[0]
            seq = ''
        else:
            seq += line[:-1]

In [67]:
err = []
def getSeq(x):
    start = int(x['start'] - 1)
    end = int(x['end'])
    if x['Gene'] not in fa.keys():
        err.append(x['Gene'])
        return
    seq = fa[x['Gene']][start:end]
    start += 1
    if os.path.exists('hmm/'+x['Gene']+'_'+str(start)):
        return
    os.mkdir('hmm/'+x['Gene']+'_'+str(start))
    with open('hmm/'+x['Gene']+'_'+str(start)+'/query.fa', 'w') as f:
        f.write('>gene\n')
        f.write(seq)
    print('cd /work/'+x['Gene']+'_'+str(start))
    print('pwd')
    n = 5
    l = len(seq)/2
    if l > 300:
        n = 2
    n = str(n)
    l = str(l)
    with open(hmdir+x['gene']+'_'+str(start)+'/run_jackhmmer.sh', 'a') as f:
        f.write('jackhmmer -N '+n+' -o log -A msa.txt  --noali --notextw -T '+l+' --domT '+l+' --incT '+l+' --incdomT '+l+' --cpu 8 query.fa /work/uniref100.fasta')

In [3]:
df.apply(getSeq, axis=1)

In [54]:
nc71 = Nc7[Nc7.raw.apply(lambda x: not x in onDomain.raw.values)]

In [57]:
lis = []
for raw in nc71.raw.drop_duplicates().values:
    group = nc71[nc71.raw == raw]
    gene = group.gene.values[0]
    pos = group.Pos.values[0]
    start = [1] + group.seq_start.values.tolist()
    end = group.seq_end.values.tolist() + [len(group.wt_seq.values[0])]
    for i in range(len(start)-1, -1, -1):
        if start[i] and start[i] <= pos:
            sp = start[i]
            break
        elif i == 0:
            sp = 1
    for i in range(0, len(end)):
        if end[i] and end[i] >= pos:
            ep = end[i]
            break
        elif i == len(end)-1:
            ep = 1
    lis.append([gene, raw, int(sp), int(ep)])
nc71pos = pd.DataFrame(lis, columns=['Gene', 'raw', 'start', 'end']).merge(pd.read_csv('Nc7.csv'), how='left')

In [58]:
nc71pos.to_csv('nc71pos.csv', index=False)

In [2]:
hmdir = 'hmm_off/'
def getSeq(x):
    start = int(x['start'])
    end = int(x['end'])
    seq = x['wt_seq'][start-1 : end]
    if os.path.exists(hmdir+x['gene']+'_'+str(start)):
        return
    os.mkdir(hmdir+x['gene']+'_'+str(start))
    with open(hmdir+x['gene']+'_'+str(start)+'/query.fa', 'w') as f:
        f.write('>'+x['gene']+'\n')
        f.write(seq)
    print('cd /work/'+x['gene']+'_'+str(start))
    print('pwd >> /work/log')
    n = 2
    l = len(seq)/2
    if l > 300:
        n = 2
    n = str(n)
    l = str(l)
    with open(hmdir+x['gene']+'_'+str(start)+'/run_jackhmmer.sh', 'a') as f:
        f.write('jackhmmer -N '+n+' -o log -A msa.txt  --noali --notextw -T '+l+' --domT '+l+' --incT '+l+' --incdomT '+l+' --cpu 8 query.fa /work/uniref100.fasta')
nc71pos.apply(getSeq, axis=1)